In [9]:
"""
01_EDA.py  —  Exploratory Data Analysis
========================================
Education Spending Analytics  |  U.S. K-12 Finance & NAEP Scores 1986-2019

Dependencies:  numpy  pandas  matplotlib  (no seaborn, no scipy)

Run from the project root:
    python notebooks/01_EDA.py

All figures are saved to  outputs/figures/eda/
"""

# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import warnings
from pathlib import Path

import matplotlib
matplotlib.use("Agg")                   # headless — swap to "TkAgg" for interactive

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
import matplotlib.cm as mcm
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# ── Paths ──────────────────────────────────────────────────────────────────
ROOT     = Path.cwd().parent 
RAW_PATH = ROOT / "Education Funding Analytics" / "data" / "raw" / "states_all.csv"
FIG_DIR  = ROOT / "Education Funding Analytics" / "outputs" / "figures" / "eda"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ── Plotting style (pure matplotlib, no seaborn) ──────────────────────────
plt.rcParams.update({
    "figure.dpi"        : 130,
    "figure.facecolor"  : "white",
    "axes.facecolor"    : "#f8f8f8",
    "axes.titlesize"    : 14,
    "axes.labelsize"    : 12,
    "axes.grid"         : True,
    "grid.color"        : "white",
    "grid.linewidth"    : 0.8,
    "legend.fontsize"   : 10,
    "xtick.labelsize"   : 10,
    "ytick.labelsize"   : 10,
})

TAB10 = [mcm.tab10(i) for i in np.linspace(0, 1, 10)]
SET2  = [mcm.Set2(i)  for i in np.linspace(0, 1,  8)]


# ── Utility helpers ────────────────────────────────────────────────────────

def save(fig: plt.Figure, name: str) -> None:
    """Save a figure and close it to free memory."""
    fig.savefig(FIG_DIR / name, bbox_inches="tight")
    plt.close(fig)
    print(f"  ✓  Saved  outputs/figures/eda/{name}")


def _palette(n: int, cmap_name: str = "tab10") -> list:
    """Return n evenly-spaced colours from any matplotlib colormap."""
    cmap = mcm.get_cmap(cmap_name)
    return [cmap(i / max(n - 1, 1)) for i in range(n)]


def _linregress(x, y):
    """
    Pure-numpy OLS returning (slope, intercept, r, p_approx, se).
    Replaces scipy.stats.linregress — p-value computed via a numpy-only
    regularised incomplete-beta implementation (accurate to ~4 d.p.).
    """
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    mask = np.isfinite(x) & np.isfinite(y)
    x, y = x[mask], y[mask]
    n = len(x)
    if n < 3:
        return np.nan, np.nan, np.nan, np.nan, np.nan

    xm, ym = x.mean(), y.mean()
    ssxx   = ((x - xm) ** 2).sum()
    ssyy   = ((y - ym) ** 2).sum()
    ssxy   = ((x - xm) * (y - ym)).sum()

    slope     = ssxy / ssxx
    intercept = ym - slope * xm
    r         = np.clip(ssxy / np.sqrt(ssxx * ssyy + 1e-300), -1, 1)
    ss_res    = ssyy - slope * ssxy
    se        = np.sqrt(max(ss_res, 0) / max((n - 2) * ssxx, 1e-300))

    t      = r * np.sqrt(n - 2) / np.sqrt(max(1 - r ** 2, 1e-12))
    df_t   = n - 2
    x_beta = df_t / (df_t + t ** 2)
    p      = _reg_inc_beta(float(x_beta), df_t / 2, 0.5)

    return slope, intercept, r, p, se


def _reg_inc_beta(x: float, a: float, b: float, max_iter: int = 200) -> float:
    """Regularised incomplete beta I_x(a,b) via Lentz continued fraction."""
    import math
    if x <= 0:
        return 0.0
    if x >= 1:
        return 1.0
    if x > (a + 1) / (a + b + 2):
        return 1.0 - _reg_inc_beta(1.0 - x, b, a, max_iter)

    lbeta = (math.lgamma(a + b) - math.lgamma(a) - math.lgamma(b)
             + a * math.log(x) + b * math.log(1.0 - x))
    front = math.exp(lbeta) / a

    tiny = 1e-300
    f = tiny
    C, D = f, 0.0
    for m in range(max_iter):
        for step in (0, 1):
            if m == 0 and step == 0:
                d = 1.0
            elif step == 0:
                d = m * (b - m) * x / ((a + 2*m - 1) * (a + 2*m))
            else:
                d = -(a + m) * (a + b + m) * x / ((a + 2*m) * (a + 2*m + 1))
            D = 1.0 + d * D
            if abs(D) < tiny:
                D = tiny
            D = 1.0 / D
            C = 1.0 + d / C
            if abs(C) < tiny:
                C = tiny
            f *= C * D
            if abs(C * D - 1.0) < 1e-8:
                return front * (f - tiny)
    return front * (f - tiny)


def _pearsonr(x, y):
    """Return (r, p) using only numpy — NaN pairs dropped automatically."""
    x, y = np.asarray(x, float), np.asarray(y, float)
    mask = np.isfinite(x) & np.isfinite(y)
    r    = np.corrcoef(x[mask], y[mask])[0, 1]
    _, _, _, p, _ = _linregress(x, y)
    return r, p


In [11]:
# =============================================================================
# LOAD  &  CLEAN
# =============================================================================
print("\n" + "="*70)
print("SECTION 1 — LOAD & CLEAN")
print("="*70)

raw = pd.read_csv(RAW_PATH)
print(f"\nRaw shape  :  {raw.shape[0]:,} rows  ×  {raw.shape[1]} columns")
print(f"Year range :  {raw['YEAR'].min()} – {raw['YEAR'].max()}")
print(f"States     :  {raw['STATE'].nunique()} unique values")

# ── Column housekeeping ────────────────────────────────────────────────────
df = raw.copy()
df.columns = df.columns.str.strip().str.upper()
df.drop(columns=["PRIMARY_KEY"], inplace=True)

NON_STATES = {"DODEA", "NATIONAL"}
df = df[~df["STATE"].str.upper().isin(NON_STATES)].copy()

NUM_COLS = [
    "ENROLL",
    "TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE",
    "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
    "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE",
    "GRADES_PK_G", "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G",
    "GRADES_1_8_G", "GRADES_9_12_G", "GRADES_ALL_G",
    "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE",
    "AVG_READING_4_SCORE", "AVG_READING_8_SCORE",
]
for col in NUM_COLS:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df["STATE"] = df["STATE"].str.strip().str.upper().astype("string")
df["YEAR"]  = df["YEAR"].astype(int)
df.drop_duplicates(inplace=True)
df.sort_values(["STATE", "YEAR"], inplace=True)
df.reset_index(drop=True, inplace=True)

print(f"Clean shape:  {df.shape[0]:,} rows  ×  {df.shape[1]} columns")
print(f"States kept:  {df['STATE'].nunique()}  (incl. DC)")

# ── Feature engineering ────────────────────────────────────────────────────
enroll    = df["ENROLL"].replace(0, np.nan)
total_rev = df["TOTAL_REVENUE"].replace(0, np.nan)
total_exp = df["TOTAL_EXPENDITURE"].replace(0, np.nan)

df["REVENUE_PER_PUPIL"]     = df["TOTAL_REVENUE"]                / enroll
df["EXPENDITURE_PER_PUPIL"] = df["TOTAL_EXPENDITURE"]            / enroll
df["INSTRUCTION_PER_PUPIL"] = df["INSTRUCTION_EXPENDITURE"]      / enroll
df["SUPPORT_PER_PUPIL"]     = df["SUPPORT_SERVICES_EXPENDITURE"] / enroll

df["PCT_FEDERAL"]     = df["FEDERAL_REVENUE"]              / total_rev * 100
df["PCT_STATE"]       = df["STATE_REVENUE"]                / total_rev * 100
df["PCT_LOCAL"]       = df["LOCAL_REVENUE"]                / total_rev * 100
df["PCT_INSTRUCTION"] = df["INSTRUCTION_EXPENDITURE"]      / total_exp * 100
df["PCT_SUPPORT"]     = df["SUPPORT_SERVICES_EXPENDITURE"] / total_exp * 100
df["PCT_CAPITAL"]     = df["CAPITAL_OUTLAY_EXPENDITURE"]   / total_exp * 100

df["AVG_MATH_SCORE"]      = df[["AVG_MATH_4_SCORE",    "AVG_MATH_8_SCORE"]].mean(axis=1, skipna=True)
df["AVG_READING_SCORE"]   = df[["AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]].mean(axis=1, skipna=True)
df["AVG_COMPOSITE_SCORE"] = df[["AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE",
                                 "AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]].mean(axis=1, skipna=True)
df["DECADE"] = (df["YEAR"] // 10 * 10).astype(str) + "s"

for col, new in [("TOTAL_EXPENDITURE","EXP_YOY"), ("TOTAL_REVENUE","REV_YOY"), ("ENROLL","ENR_YOY")]:
    df[new] = df.groupby("STATE")[col].pct_change() * 100

print("\nFeature engineering complete.")

# ── FIG 1-A  |  Missing-value bar chart ───────────────────────────────────
print("\n── Fig 1-A  Missing-value heatmap")

miss_pct  = df.isnull().mean().sort_values(ascending=False) * 100
miss_pct  = miss_pct[miss_pct > 0]
bar_colors = ["#2ecc71" if v < 10 else "#f39c12" if v < 40 else "#e74c3c"
              for v in miss_pct.values]

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(miss_pct.index[::-1], miss_pct.values[::-1], color=bar_colors[::-1])
ax.axvline(10, color="#f39c12", linestyle="--", linewidth=1, alpha=0.8, label="10%")
ax.axvline(40, color="#e74c3c", linestyle="--", linewidth=1, alpha=0.8, label="40%")
ax.set_xlabel("% Missing")
ax.set_title("Missing Data by Column")
ax.legend()
fig.tight_layout()
save(fig, "01a_missing_values.png")

# ── FIG 1-B  |  Data coverage by year ─────────────────────────────────────
print("── Fig 1-B  Data coverage by year")

coverage = df.groupby("YEAR")[["TOTAL_EXPENDITURE", "AVG_COMPOSITE_SCORE", "ENROLL"]].count()
coverage.columns = ["Financial data", "NAEP Scores", "Enrollment"]

fig, ax = plt.subplots(figsize=(13, 4))
for col, color in zip(coverage.columns, TAB10):
    ax.plot(coverage.index, coverage[col], marker="o", markersize=3,
            linewidth=2, color=color, label=col)
ax.set_xlabel("Year")
ax.set_ylabel("# States with data")
ax.set_title("Data Coverage Over Time (out of 51 states + DC)")
ax.legend(loc="lower right")
fig.tight_layout()
save(fig, "01b_data_coverage.png")


SECTION 1 — LOAD & CLEAN

Raw shape  :  1,715 rows  ×  25 columns
Year range :  1986 – 2019
States     :  53 unique values
Clean shape:  1,683 rows  ×  24 columns
States kept:  51  (incl. DC)

Feature engineering complete.

── Fig 1-A  Missing-value heatmap
  ✓  Saved  outputs/figures/eda/01a_missing_values.png
── Fig 1-B  Data coverage by year
  ✓  Saved  outputs/figures/eda/01b_data_coverage.png


In [13]:
# =============================================================================
# DISTRIBUTIONS  &  CORRELATIONS
# =============================================================================
print("\n" + "="*70)
print("SECTION 2 — DISTRIBUTIONS & CORRELATIONS")
print("="*70)

# ── FIG 2-A  |  Per-pupil expenditure distributions by decade ─────────────
print("\n── Fig 2-A  Per-pupil expenditure distributions by decade")

decades  = sorted(df["DECADE"].dropna().unique())
pal_dec  = _palette(len(decades), "tab10")

fig, axes = plt.subplots(1, len(decades), figsize=(16, 5), sharey=False)
for ax, dec, color in zip(axes, decades, pal_dec):
    sub = df[df["DECADE"] == dec]["EXPENDITURE_PER_PUPIL"].dropna()
    ax.hist(sub, bins=25, color=color, edgecolor="white", alpha=0.85)
    ax.axvline(sub.median(), color="black", linestyle="--", linewidth=1.5,
               label=f"Median\n${sub.median():,.0f}")
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x/1000:.0f}k"))
    ax.set_title(dec)
    ax.set_xlabel("Expenditure / Pupil")
    ax.legend(fontsize=8)
axes[0].set_ylabel("Count")
fig.suptitle("Distribution of Per-Pupil Expenditure by Decade", y=1.02, fontsize=15)
fig.tight_layout()
save(fig, "02a_expenditure_distributions_by_decade.png")

# ── FIG 2-B  |  NAEP score distributions ─────────────────────────────────
print("── Fig 2-B  NAEP score distributions")

score_cols_map = {
    "AVG_MATH_4_SCORE"   : "Math  Gr.4",
    "AVG_MATH_8_SCORE"   : "Math  Gr.8",
    "AVG_READING_4_SCORE": "Reading Gr.4",
    "AVG_READING_8_SCORE": "Reading Gr.8",
}
fig, axes = plt.subplots(1, 4, figsize=(16, 5))
for ax, (col, label) in zip(axes, score_cols_map.items()):
    data = df[col].dropna()
    ax.hist(data, bins=30, color=TAB10[0], edgecolor="white", alpha=0.85)
    ax.axvline(data.mean(),   color="black",   linestyle="-",  linewidth=1.5,
               label=f"Mean {data.mean():.1f}")
    ax.axvline(data.median(), color="#e74c3c", linestyle="--", linewidth=1.5,
               label=f"Median {data.median():.1f}")
    ax.set_title(label)
    ax.set_xlabel("NAEP Scale Score")
    ax.legend(fontsize=8)
axes[0].set_ylabel("Count")
fig.suptitle("Distribution of NAEP Test Scores (all states, all years)", y=1.02, fontsize=15)
fig.tight_layout()
save(fig, "02b_naep_score_distributions.png")

# ── FIG 2-C  |  Revenue composition box plots ─────────────────────────────
print("── Fig 2-C  Revenue composition box plots")

rev_groups = {
    "Federal": df["PCT_FEDERAL"].dropna().values,
    "State"  : df["PCT_STATE"].dropna().values,
    "Local"  : df["PCT_LOCAL"].dropna().values,
}
fig, ax = plt.subplots(figsize=(8, 5))
bp = ax.boxplot(
    list(rev_groups.values()),
    labels=list(rev_groups.keys()),
    patch_artist=True,
    medianprops=dict(color="black", linewidth=2),
    widths=0.45,
)
for patch, color in zip(bp["boxes"], ["#4C72B0", "#55A868", "#C44E52"]):
    patch.set_facecolor(color)
    patch.set_alpha(0.75)
ax.yaxis.set_major_formatter(mticker.PercentFormatter())
ax.set_ylabel("% of Total Revenue")
ax.set_title("Revenue Source Composition — All States, All Years")
fig.tight_layout()
save(fig, "02c_revenue_composition_boxplot.png")

# ── FIG 2-D  |  Expenditure breakdown stacked bar ─────────────────────────
print("── Fig 2-D  Expenditure breakdown stacked bar")

exp_breakdown = df.groupby("YEAR")[["PCT_INSTRUCTION","PCT_SUPPORT","PCT_CAPITAL"]].median().dropna()
colors_exp = ["#4C72B0", "#55A868", "#DD8452"]
labels_exp = ["Instruction", "Support Services", "Capital Outlay"]

fig, ax = plt.subplots(figsize=(14, 5))
bottom = np.zeros(len(exp_breakdown))
for col, color, label in zip(exp_breakdown.columns, colors_exp, labels_exp):
    ax.bar(exp_breakdown.index, exp_breakdown[col], bottom=bottom,
           color=color, label=label, width=0.8)
    bottom += exp_breakdown[col].values
ax.yaxis.set_major_formatter(mticker.PercentFormatter())
ax.set_xlabel("Year")
ax.set_ylabel("% of Total Expenditure")
ax.set_title("National Median Expenditure Breakdown by Category Over Time")
ax.legend(loc="upper right")
ax.tick_params(axis="x", rotation=45)
fig.tight_layout()
save(fig, "02d_expenditure_breakdown_stacked_bar.png")

# ── FIG 2-E  |  Correlation heatmap (pure matplotlib imshow) ──────────────
print("── Fig 2-E  Correlation heatmap")

CORR_COLS = [c for c in [
    "EXPENDITURE_PER_PUPIL", "REVENUE_PER_PUPIL",
    "PCT_FEDERAL", "PCT_STATE", "PCT_LOCAL",
    "PCT_INSTRUCTION", "PCT_SUPPORT", "PCT_CAPITAL",
    "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE",
    "AVG_READING_4_SCORE", "AVG_READING_8_SCORE",
    "AVG_COMPOSITE_SCORE", "ENROLL",
] if c in df.columns]

corr   = df[CORR_COLS].corr().values
n_c    = len(CORR_COLS)
labels = [c.replace("_SCORE","").replace("AVG_","").replace("_"," ") for c in CORR_COLS]

# Mask upper triangle
mask   = np.triu(np.ones((n_c, n_c), dtype=bool), k=1)
masked = np.where(mask, np.nan, corr)

cmap_h = mcm.RdYlGn
cmap_h.set_bad("white")

fig, ax = plt.subplots(figsize=(13, 11))
im = ax.imshow(masked, cmap=cmap_h, vmin=-1, vmax=1, aspect="auto")
for i in range(n_c):
    for j in range(n_c):
        if not mask[i, j]:
            val = corr[i, j]
            ax.text(j, i, f"{val:.2f}", ha="center", va="center",
                    fontsize=7.5, color="black" if abs(val) < 0.6 else "white")
ax.set_xticks(range(n_c));  ax.set_xticklabels(labels, rotation=45, ha="right", fontsize=8)
ax.set_yticks(range(n_c));  ax.set_yticklabels(labels, fontsize=8)
fig.colorbar(im, ax=ax, fraction=0.025, pad=0.02)
ax.set_title("Correlation Heatmap — Key Metrics (lower triangle)", pad=12)
fig.tight_layout()
save(fig, "02e_correlation_heatmap.png")

# ── FIG 2-F  |  Score vs. spending scatter ────────────────────────────────
print("── Fig 2-F  Score vs. spending scatter")

scatter_df  = df[["EXPENDITURE_PER_PUPIL","AVG_COMPOSITE_SCORE","DECADE"]].dropna()
pal_dec_map = {dec: pal_dec[i] for i, dec in enumerate(decades)}

fig, ax = plt.subplots(figsize=(10, 7))
for decade in sorted(scatter_df["DECADE"].unique()):
    sub = scatter_df[scatter_df["DECADE"] == decade]
    ax.scatter(sub["EXPENDITURE_PER_PUPIL"], sub["AVG_COMPOSITE_SCORE"],
               label=decade, alpha=0.55, s=35, color=pal_dec_map.get(decade, "grey"))

x_all = scatter_df["EXPENDITURE_PER_PUPIL"].values
y_all = scatter_df["AVG_COMPOSITE_SCORE"].values
r, p  = _pearsonr(x_all, y_all)
ok    = np.isfinite(x_all) & np.isfinite(y_all)
m, b  = np.polyfit(x_all[ok], y_all[ok], 1)
x_ln  = np.linspace(np.nanmin(x_all), np.nanmax(x_all), 200)
ax.plot(x_ln, m * x_ln + b, color="black", linewidth=1.8, linestyle="--",
        label=f"OLS  r={r:.2f}  p={p:.3f}")
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax.set_xlabel("Per-Pupil Expenditure")
ax.set_ylabel("Avg Composite NAEP Score")
ax.set_title("Test Score vs. Per-Pupil Spending (by Decade)")
ax.legend(loc="lower right", ncol=2)
fig.tight_layout()
save(fig, "02f_score_vs_spending_scatter.png")

# ── FIG 2-G  |  State rankings bar chart ─────────────────────────────────
print("── Fig 2-G  State-level per-pupil expenditure (latest year)")

latest_year = int(df["YEAR"].max())
latest      = df[df["YEAR"] == latest_year][["STATE","EXPENDITURE_PER_PUPIL"]].dropna()
latest      = latest.sort_values("EXPENDITURE_PER_PUPIL", ascending=True)

cmap_rg  = mcm.RdYlGn
bar_norm = mcolors.Normalize(vmin=latest["EXPENDITURE_PER_PUPIL"].min(),
                              vmax=latest["EXPENDITURE_PER_PUPIL"].max())
bar_cols = [cmap_rg(bar_norm(v)) for v in latest["EXPENDITURE_PER_PUPIL"]]

fig, ax = plt.subplots(figsize=(10, 14))
ax.barh(latest["STATE"], latest["EXPENDITURE_PER_PUPIL"], color=bar_cols)
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax.set_xlabel("Per-Pupil Expenditure")
ax.set_title(f"Per-Pupil Expenditure by State  ({latest_year})")
fig.tight_layout()
save(fig, "02g_state_expenditure_ranking.png")


SECTION 2 — DISTRIBUTIONS & CORRELATIONS

── Fig 2-A  Per-pupil expenditure distributions by decade
  ✓  Saved  outputs/figures/eda/02a_expenditure_distributions_by_decade.png
── Fig 2-B  NAEP score distributions
  ✓  Saved  outputs/figures/eda/02b_naep_score_distributions.png
── Fig 2-C  Revenue composition box plots
  ✓  Saved  outputs/figures/eda/02c_revenue_composition_boxplot.png
── Fig 2-D  Expenditure breakdown stacked bar
  ✓  Saved  outputs/figures/eda/02d_expenditure_breakdown_stacked_bar.png
── Fig 2-E  Correlation heatmap
  ✓  Saved  outputs/figures/eda/02e_correlation_heatmap.png
── Fig 2-F  Score vs. spending scatter
  ✓  Saved  outputs/figures/eda/02f_score_vs_spending_scatter.png
── Fig 2-G  State-level per-pupil expenditure (latest year)
  ✓  Saved  outputs/figures/eda/02g_state_expenditure_ranking.png


In [15]:
# =============================================================================
# TIME-SERIES INSIGHTS
# =============================================================================
print("\n" + "="*70)
print("SECTION 3 — TIME-SERIES INSIGHTS")
print("="*70)

# ── FIG 3-A  |  National median spending trend ────────────────────────────
print("\n── Fig 3-A  National median spending trend")

SPEND_COLS = {
    "EXPENDITURE_PER_PUPIL" : "Total Expenditure",
    "INSTRUCTION_PER_PUPIL" : "Instruction",
    "SUPPORT_PER_PUPIL"     : "Support Services",
}
national_med = df.groupby("YEAR")[list(SPEND_COLS)].median()

fig, ax = plt.subplots(figsize=(13, 5))
for (col, label), color in zip(SPEND_COLS.items(), TAB10):
    ax.plot(national_med.index, national_med[col], linewidth=2.2,
            marker="o", markersize=3, label=label, color=color)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax.set_xlabel("Year")
ax.set_ylabel("Per-Pupil Expenditure (USD)")
ax.set_title("National Median Per-Pupil Spending Trend  (1986–2019)")
ax.legend()
fig.tight_layout()
save(fig, "03a_national_spending_trend.png")

# ── FIG 3-B  |  NAEP score trends ────────────────────────────────────────
print("── Fig 3-B  NAEP score trends")

score_trend = df.groupby("YEAR")[list(score_cols_map)].median()

fig, ax = plt.subplots(figsize=(13, 5))
for (col, label), color in zip(score_cols_map.items(), TAB10):
    ax.plot(score_trend.index, score_trend[col], linewidth=2.2,
            marker="o", markersize=3, label=label, color=color)
ax.set_xlabel("Year")
ax.set_ylabel("NAEP Scale Score")
ax.set_title("National Median NAEP Scores Over Time")
ax.legend()
fig.tight_layout()
save(fig, "03b_naep_score_trends.png")

# ── FIG 3-C  |  Revenue source composition shift ─────────────────────────
print("── Fig 3-C  Revenue composition shift over time")

rev_trend = df.groupby("YEAR")[["PCT_FEDERAL","PCT_STATE","PCT_LOCAL"]].median().dropna()

fig, ax = plt.subplots(figsize=(13, 5))
ax.stackplot(
    rev_trend.index,
    rev_trend["PCT_LOCAL"], rev_trend["PCT_STATE"], rev_trend["PCT_FEDERAL"],
    labels=["Local", "State", "Federal"],
    colors=["#C44E52", "#55A868", "#4C72B0"], alpha=0.85,
)
ax.yaxis.set_major_formatter(mticker.PercentFormatter())
ax.set_xlabel("Year")
ax.set_ylabel("% of Total Revenue")
ax.set_title("Revenue Source Composition Shift (National Median)  1986–2019")
ax.legend(loc="upper left")
fig.tight_layout()
save(fig, "03c_revenue_composition_shift.png")

# ── FIG 3-D  |  YoY growth rate distributions ────────────────────────────
print("── Fig 3-D  YoY growth rate distributions")

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
for ax, col, label, color in zip(
    axes,
    ["EXP_YOY", "REV_YOY"],
    ["Expenditure YoY %", "Revenue YoY %"],
    [TAB10[0], TAB10[2]],
):
    data = df[col].dropna()
    data = data[(data >= data.quantile(0.01)) & (data <= data.quantile(0.99))]
    ax.hist(data, bins=40, color=color, edgecolor="white", alpha=0.85)
    ax.axvline(0, color="black", linewidth=1.2, linestyle="--")
    ax.axvline(data.mean(), color="#e74c3c", linewidth=1.5,
               label=f"Mean {data.mean():.1f}%")
    ax.set_title(label)
    ax.set_xlabel("Year-over-Year Change (%)")
    ax.legend()
axes[0].set_ylabel("Count")
fig.suptitle("Distribution of Annual Growth Rates  (all states, trimmed ±1%)", fontsize=14)
fig.tight_layout()
save(fig, "03d_yoy_growth_distributions.png")

# ── FIG 3-E  |  Spotlight — 6 large states ───────────────────────────────
print("── Fig 3-E  State spotlight — spending & scores")

SPOTLIGHT = [s for s in ["CALIFORNIA","TEXAS","NEW_YORK","FLORIDA","ILLINOIS","PENNSYLVANIA"]
             if s in df["STATE"].values]
pal6 = _palette(len(SPOTLIGHT), "tab10")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(13, 9), sharex=True)
for state, color in zip(SPOTLIGHT, pal6):
    sub = df[df["STATE"] == state].sort_values("YEAR")
    ax1.plot(sub["YEAR"], sub["EXPENDITURE_PER_PUPIL"], linewidth=2,
             marker="o", markersize=3, color=color,
             label=state.replace("_"," ").title())
    s2 = sub.dropna(subset=["AVG_COMPOSITE_SCORE"])
    ax2.plot(s2["YEAR"], s2["AVG_COMPOSITE_SCORE"], linewidth=2,
             marker="o", markersize=3, color=color)
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax1.set_ylabel("Per-Pupil Expenditure")
ax1.set_title("Per-Pupil Expenditure")
ax1.legend(ncol=2)
ax2.set_ylabel("Avg Composite NAEP Score")
ax2.set_xlabel("Year")
ax2.set_title("Composite NAEP Score")
fig.suptitle("Large-State Spotlight  (1986–2019)", fontsize=15, y=1.01)
fig.tight_layout()
save(fig, "03e_state_spotlight.png")

# ── FIG 3-F  |  Rolling 5-yr average — top vs. bottom spenders ───────────
print("── Fig 3-F  Rolling 5-year average — top vs. bottom spenders")

mean_spend = (df.groupby("STATE")["EXPENDITURE_PER_PUPIL"].mean()
              .dropna().sort_values(ascending=False))
TOP5, BOTTOM5 = mean_spend.head(5).index.tolist(), mean_spend.tail(5).index.tolist()

fig, ax = plt.subplots(figsize=(13, 6))
for state, color in zip(TOP5, _palette(5, "Blues_r")):
    sub  = df[df["STATE"] == state].sort_values("YEAR").set_index("YEAR")
    roll = sub["EXPENDITURE_PER_PUPIL"].rolling(5, min_periods=3).mean()
    ax.plot(roll.index, roll.values, linewidth=2.2, linestyle="-", color=color,
            label=state.replace("_"," ").title() + " (top)")
for state, color in zip(BOTTOM5, _palette(5, "Reds_r")):
    sub  = df[df["STATE"] == state].sort_values("YEAR").set_index("YEAR")
    roll = sub["EXPENDITURE_PER_PUPIL"].rolling(5, min_periods=3).mean()
    ax.plot(roll.index, roll.values, linewidth=2.2, linestyle="--", color=color,
            label=state.replace("_"," ").title() + " (bot)")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax.set_xlabel("Year")
ax.set_ylabel("5-yr Rolling Per-Pupil Expenditure")
ax.set_title("5-Year Rolling Average Expenditure — Top-5 vs. Bottom-5 Spenders")
ax.legend(ncol=2, fontsize=9)
fig.tight_layout()
save(fig, "03f_rolling_top_vs_bottom_spenders.png")


SECTION 3 — TIME-SERIES INSIGHTS

── Fig 3-A  National median spending trend
  ✓  Saved  outputs/figures/eda/03a_national_spending_trend.png
── Fig 3-B  NAEP score trends
  ✓  Saved  outputs/figures/eda/03b_naep_score_trends.png
── Fig 3-C  Revenue composition shift over time
  ✓  Saved  outputs/figures/eda/03c_revenue_composition_shift.png
── Fig 3-D  YoY growth rate distributions
  ✓  Saved  outputs/figures/eda/03d_yoy_growth_distributions.png
── Fig 3-E  State spotlight — spending & scores
  ✓  Saved  outputs/figures/eda/03e_state_spotlight.png
── Fig 3-F  Rolling 5-year average — top vs. bottom spenders
  ✓  Saved  outputs/figures/eda/03f_rolling_top_vs_bottom_spenders.png


In [16]:
# =============================================================================
# GEOSPATIAL ANALYSIS
# =============================================================================
print("\n" + "="*70)
print("SECTION 4A — GEOSPATIAL ANALYSIS")
print("="*70)

STATE_ABBR = {
    "ALABAMA":"AL","ALASKA":"AK","ARIZONA":"AZ","ARKANSAS":"AR","CALIFORNIA":"CA",
    "COLORADO":"CO","CONNECTICUT":"CT","DELAWARE":"DE","DISTRICT_OF_COLUMBIA":"DC",
    "FLORIDA":"FL","GEORGIA":"GA","HAWAII":"HI","IDAHO":"ID","ILLINOIS":"IL",
    "INDIANA":"IN","IOWA":"IA","KANSAS":"KS","KENTUCKY":"KY","LOUISIANA":"LA",
    "MAINE":"ME","MARYLAND":"MD","MASSACHUSETTS":"MA","MICHIGAN":"MI",
    "MINNESOTA":"MN","MISSISSIPPI":"MS","MISSOURI":"MO","MONTANA":"MT",
    "NEBRASKA":"NE","NEVADA":"NV","NEW_HAMPSHIRE":"NH","NEW_JERSEY":"NJ",
    "NEW_MEXICO":"NM","NEW_YORK":"NY","NORTH_CAROLINA":"NC","NORTH_DAKOTA":"ND",
    "OHIO":"OH","OKLAHOMA":"OK","OREGON":"OR","PENNSYLVANIA":"PA","RHODE_ISLAND":"RI",
    "SOUTH_CAROLINA":"SC","SOUTH_DAKOTA":"SD","TENNESSEE":"TN","TEXAS":"TX",
    "UTAH":"UT","VERMONT":"VT","VIRGINIA":"VA","WASHINGTON":"WA",
    "WEST_VIRGINIA":"WV","WISCONSIN":"WI","WYOMING":"WY",
}
STATE_COORDS = {
    "AL":(-86.8,32.8),"AK":(-153.4,64.2),"AZ":(-111.1,34.3),"AR":(-92.4,34.9),
    "CA":(-119.4,37.2),"CO":(-105.5,39.0),"CT":(-72.7,41.6),"DE":(-75.5,39.0),
    "DC":(-77.0,38.9),"FL":(-81.5,27.8),"GA":(-83.4,32.7),"HI":(-157.8,20.3),
    "ID":(-114.5,44.4),"IL":(-89.2,40.0),"IN":(-86.3,40.3),"IA":(-93.6,42.0),
    "KS":(-98.4,38.5),"KY":(-84.3,37.5),"LA":(-91.8,31.1),"ME":(-69.4,44.7),
    "MD":(-76.6,39.0),"MA":(-71.5,42.4),"MI":(-85.4,44.3),"MN":(-94.3,46.4),
    "MS":(-89.7,32.7),"MO":(-92.5,38.3),"MT":(-110.5,47.0),"NE":(-99.9,41.5),
    "NV":(-117.1,38.5),"NH":(-71.6,43.7),"NJ":(-74.7,40.1),"NM":(-106.1,34.5),
    "NY":(-75.5,43.0),"NC":(-79.8,35.5),"ND":(-100.5,47.5),"OH":(-82.8,40.4),
    "OK":(-97.5,35.5),"OR":(-120.6,44.0),"PA":(-77.2,40.9),"RI":(-71.5,41.7),
    "SC":(-80.9,33.8),"SD":(-100.2,44.4),"TN":(-86.7,35.9),"TX":(-99.3,31.5),
    "UT":(-111.1,39.3),"VT":(-72.7,44.1),"VA":(-78.7,37.5),"WA":(-120.5,47.4),
    "WV":(-80.6,38.6),"WI":(-89.6,44.3),"WY":(-107.5,43.0),
}

df["ABBR"] = df["STATE"].map(STATE_ABBR)


def make_bubble_map(snapshot, metric, title, filename, cmap="RdYlGn"):
    snap = snapshot.copy()
    snap["ABBR"] = snap["STATE"].map(STATE_ABBR)
    snap = snap.dropna(subset=[metric, "ABBR"])
    snap["LON"] = snap["ABBR"].map(lambda a: STATE_COORDS.get(a, (np.nan,np.nan))[0])
    snap["LAT"] = snap["ABBR"].map(lambda a: STATE_COORDS.get(a, (np.nan,np.nan))[1])
    snap = snap.dropna(subset=["LON","LAT"])

    vals      = snap[metric].values
    norm_map  = mcolors.Normalize(vmin=np.nanpercentile(vals, 5),
                                   vmax=np.nanpercentile(vals, 95))
    fig, ax = plt.subplots(figsize=(14, 8))
    sc = ax.scatter(snap["LON"], snap["LAT"],
                    c=vals, cmap=cmap, norm=norm_map,
                    s=220, edgecolors="white", linewidths=0.8, zorder=3)
    for _, row in snap.iterrows():
        ax.text(row["LON"], row["LAT"], row["ABBR"],
                ha="center", va="center", fontsize=6.5,
                fontweight="bold", color="black", zorder=4)
    cb = fig.colorbar(sc, ax=ax, fraction=0.025, pad=0.02)
    cb.set_label(metric.replace("_"," ").title())
    ax.set_xlim(-130, -65);  ax.set_ylim(23, 52)
    ax.set_facecolor("#cce5ff")
    ax.set_xlabel("Longitude");  ax.set_ylabel("Latitude")
    ax.set_title(title)
    fig.tight_layout()
    save(fig, filename)


snap_latest = df[df["YEAR"] == latest_year]
snap_early  = df[df["YEAR"] == 1995]

print("\n── Fig 4-A  Per-pupil expenditure map (latest year)")
make_bubble_map(snap_latest, "EXPENDITURE_PER_PUPIL",
                f"Per-Pupil Expenditure by State  ({latest_year})",
                "04a_geo_expenditure_latest.png")

print("── Fig 4-B  NAEP composite score map (latest year)")
make_bubble_map(snap_latest, "AVG_COMPOSITE_SCORE",
                f"Avg Composite NAEP Score by State  ({latest_year})",
                "04b_geo_naep_score_latest.png")

print("── Fig 4-C  Change in per-pupil spending map")
early     = snap_early[["STATE","EXPENDITURE_PER_PUPIL"]].rename(
    columns={"EXPENDITURE_PER_PUPIL":"EXP_1995"})
change_df = snap_latest[["STATE","EXPENDITURE_PER_PUPIL"]].merge(early, on="STATE")
change_df["EXP_CHANGE_PCT"] = ((change_df["EXPENDITURE_PER_PUPIL"] - change_df["EXP_1995"])
                                / change_df["EXP_1995"] * 100)
make_bubble_map(change_df, "EXP_CHANGE_PCT",
                f"% Change in Per-Pupil Expenditure  (1995 → {latest_year})",
                "04c_geo_expenditure_change.png")

print("── Fig 4-D  Federal revenue dependency map")
make_bubble_map(snap_latest, "PCT_FEDERAL",
                f"Federal Revenue Share by State  ({latest_year})",
                "04d_geo_federal_dependency.png", cmap="YlOrRd")

print("── Fig 4-E  Spending efficiency map")
snap_eff = snap_latest.copy()
snap_eff["SCORE_PER_1K"] = (snap_eff["AVG_COMPOSITE_SCORE"]
                             / (snap_eff["EXPENDITURE_PER_PUPIL"] / 1000))
make_bubble_map(snap_eff, "SCORE_PER_1K",
                f"NAEP Score per $1k Per-Pupil Expenditure  ({latest_year})",
                "04e_geo_spending_efficiency.png")


# =============================================================================
# MEAN-REVERSION  &  CONVERGENCE
# =============================================================================
print("\n" + "="*70)
print("SECTION 4B — MEAN-REVERSION & CONVERGENCE")
print("="*70)

# ── FIG 4-F  |  σ-convergence ─────────────────────────────────────────────
print("\n── Fig 4-F  Sigma-convergence in per-pupil spending")

sigma = df.groupby("YEAR")["EXPENDITURE_PER_PUPIL"].std()
cv    = df.groupby("YEAR")["EXPENDITURE_PER_PUPIL"].apply(lambda x: x.std() / x.mean())

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
ax1.plot(sigma.index, sigma.values, linewidth=2.2, color=TAB10[0], marker="o", markersize=3)
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
ax1.set_ylabel("Std Dev of Expenditure/Pupil")
ax1.set_title("σ-Convergence: Cross-State Dispersion of Per-Pupil Expenditure")
ax2.plot(cv.index, cv.values * 100, linewidth=2.2, color="#C44E52", marker="o", markersize=3)
ax2.yaxis.set_major_formatter(mticker.PercentFormatter())
ax2.set_ylabel("Coefficient of Variation (%)")
ax2.set_xlabel("Year")
ax2.set_title("Coefficient of Variation (normalised dispersion)")
fig.tight_layout()
save(fig, "04f_sigma_convergence.png")

# ── FIG 4-G  |  β-convergence ─────────────────────────────────────────────
print("── Fig 4-G  Beta-convergence in per-pupil spending")

BASE_YEAR, END_YEAR = 1995, latest_year
base_   = df[df["YEAR"] == BASE_YEAR][["STATE","EXPENDITURE_PER_PUPIL"]].dropna()
end_    = df[df["YEAR"] == END_YEAR ][["STATE","EXPENDITURE_PER_PUPIL"]].dropna()
beta_df = base_.merge(end_, on="STATE", suffixes=("_base","_end"))
beta_df["LOG_BASE"]     = np.log(beta_df["EXPENDITURE_PER_PUPIL_base"])
beta_df["TOTAL_GROWTH"] = (np.log(beta_df["EXPENDITURE_PER_PUPIL_end"])
                            - np.log(beta_df["EXPENDITURE_PER_PUPIL_base"]))

slope, intercept, r_val, p_val, _ = _linregress(
    beta_df["LOG_BASE"].values, beta_df["TOTAL_GROWTH"].values)

fig, ax = plt.subplots(figsize=(10, 7))
ax.scatter(beta_df["LOG_BASE"], beta_df["TOTAL_GROWTH"],
           color=TAB10[0], s=60, alpha=0.75, edgecolors="white")
for _, row in beta_df.iterrows():
    abbr = STATE_ABBR.get(row["STATE"], "")
    ax.annotate(abbr, (row["LOG_BASE"], row["TOTAL_GROWTH"]),
                xytext=(3, 2), textcoords="offset points", fontsize=7, color="#555")
if np.isfinite(slope):
    x_ln  = np.linspace(beta_df["LOG_BASE"].min(), beta_df["LOG_BASE"].max(), 100)
    p_str = f"{p_val:.4f}" if np.isfinite(p_val) else "n/a"
    ax.plot(x_ln, intercept + slope * x_ln, color="#e74c3c", linewidth=2,
            label=f"β = {slope:.3f}   r² = {r_val**2:.3f}   p = {p_str}")
ax.axhline(0, color="black", linewidth=0.8, linestyle=":")
ax.set_xlabel(f"Log Per-Pupil Expenditure  ({BASE_YEAR})")
ax.set_ylabel(f"Log Growth in Expenditure  ({BASE_YEAR}→{END_YEAR})")
ax.set_title(f"β-Convergence in Per-Pupil Expenditure  ({BASE_YEAR}–{END_YEAR})\n"
             f"Negative β ⟹ lower-spending states grew faster (converging)")
ax.legend()
fig.tight_layout()
save(fig, "04g_beta_convergence.png")

if np.isfinite(slope):
    print(f"   β = {slope:.4f}  →  {'converging (β<0)' if slope < 0 else 'diverging (β>0)'}")

# ── FIG 4-H  |  Mean-reversion half-lives ────────────────────────────────
print("── Fig 4-H  Mean-reversion speed — deviation half-lives")

nat_med_ts = df.groupby("YEAR")["EXPENDITURE_PER_PUPIL"].transform("median")
df["EXP_DEV"] = df["EXPENDITURE_PER_PUPIL"] - nat_med_ts

half_lives = {}
for state, grp in df.groupby("STATE"):
    dev = grp.sort_values("YEAR")["EXP_DEV"].dropna()
    if len(dev) < 8:
        continue
    y_r = dev.diff().dropna().values
    x_r = dev.shift(1).dropna().values
    if len(x_r) < 6:
        continue
    try:
        s_ar, _, _, p_ar, _ = _linregress(x_r, y_r)
        if np.isfinite(s_ar) and s_ar < 0 and (not np.isfinite(p_ar) or p_ar < 0.2):
            hl = -np.log(2) / np.log(1 + s_ar)
            if 0 < hl < 100:
                half_lives[state] = hl
    except Exception:
        continue

if half_lives:
    hl_s = pd.Series(half_lives).sort_values()
    c_hl = ["#2ecc71" if v < 10 else "#f39c12" if v < 20 else "#e74c3c" for v in hl_s.values]
    fig, ax = plt.subplots(figsize=(10, max(5, len(hl_s) * 0.35)))
    ax.barh(hl_s.index.str.replace("_"," ").str.title(), hl_s.values, color=c_hl)
    ax.axvline(hl_s.median(), color="black", linestyle="--", linewidth=1.5,
               label=f"Median ≈ {hl_s.median():.1f} yrs")
    ax.set_xlabel("Half-Life of Deviation from National Median (years)")
    ax.set_title("Mean-Reversion Speed: Per-Pupil Expenditure Deviations\n"
                 "(States where AR(1) coefficient is significant at p < 0.20)")
    ax.legend()
    fig.tight_layout()
    save(fig, "04h_mean_reversion_halflives.png")
    print(f"   Median half-life ≈ {hl_s.median():.1f} years  across {len(hl_s)} states")
else:
    print("   Insufficient data for half-life estimation.")

# ── FIG 4-I  |  Fan chart ────────────────────────────────────────────────
print("── Fig 4-I  Fan chart — spending dispersion over time")

pct_from_median = (
    df.groupby(["YEAR","STATE"])["EXPENDITURE_PER_PUPIL"].mean()
    .unstack("STATE")
    .apply(lambda row: (row / row.median() - 1) * 100, axis=1)
)
p10 = pct_from_median.quantile(0.10, axis=1)
p25 = pct_from_median.quantile(0.25, axis=1)
p75 = pct_from_median.quantile(0.75, axis=1)
p90 = pct_from_median.quantile(0.90, axis=1)
yrs = pct_from_median.index

NAMED   = ["NEW_YORK","MISSISSIPPI","WYOMING","CALIFORNIA","UTAH"]
pal_fan = _palette(len(NAMED), "Set2")

fig, ax = plt.subplots(figsize=(13, 6))
ax.fill_between(yrs, p10, p90, alpha=0.18, color=TAB10[0], label="P10–P90")
ax.fill_between(yrs, p25, p75, alpha=0.35, color=TAB10[0], label="P25–P75")
ax.axhline(0, color="black", linewidth=1.8, label="National Median")
for state, color in zip(NAMED, pal_fan):
    if state in pct_from_median.columns:
        ax.plot(yrs, pct_from_median[state], linewidth=1.8, color=color,
                label=state.replace("_"," ").title())
ax.yaxis.set_major_formatter(mticker.PercentFormatter())
ax.set_xlabel("Year")
ax.set_ylabel("% Deviation from National Median")
ax.set_title("Per-Pupil Expenditure — Fan Chart Relative to National Median")
ax.legend(loc="upper left", ncol=2, fontsize=9)
fig.tight_layout()
save(fig, "04i_spending_fan_chart.png")

# ── FIG 4-J  |  Score gap — top vs. bottom spending quartile ─────────────
print("── Fig 4-J  Score gap — top vs. bottom spending quartile")


def _safe_qcut(x):
    try:
        return pd.qcut(x, 4, labels=["Q1 (Low)","Q2","Q3","Q4 (High)"], duplicates="drop")
    except ValueError:
        return pd.Series([np.nan] * len(x), index=x.index)


df["SPEND_QUARTILE"] = df.groupby("YEAR")["EXPENDITURE_PER_PUPIL"].transform(_safe_qcut)

q_scores = (
    df[df["SPEND_QUARTILE"].isin(["Q1 (Low)","Q4 (High)"])]
    .groupby(["YEAR","SPEND_QUARTILE"])["AVG_COMPOSITE_SCORE"]
    .median()
    .unstack("SPEND_QUARTILE")
    .dropna()
)

if "Q1 (Low)" in q_scores.columns and "Q4 (High)" in q_scores.columns:
    q_scores["GAP"] = q_scores["Q4 (High)"] - q_scores["Q1 (Low)"]
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 9), sharex=True)
    ax1.plot(q_scores.index, q_scores["Q1 (Low)"], linewidth=2.2,
             color="#e74c3c", marker="o", markersize=3, label="Q1 Low Spend")
    ax1.plot(q_scores.index, q_scores["Q4 (High)"], linewidth=2.2,
             color="#2ecc71", marker="o", markersize=3, label="Q4 High Spend")
    ax1.set_ylabel("Median Composite NAEP Score")
    ax1.set_title("Composite NAEP Score by Spending Quartile")
    ax1.legend()
    ax2.fill_between(q_scores.index, q_scores["GAP"], alpha=0.4, color=TAB10[0])
    ax2.plot(q_scores.index, q_scores["GAP"], linewidth=2.2,
             color=TAB10[0], marker="o", markersize=3)
    ax2.axhline(q_scores["GAP"].mean(), color="black", linestyle="--", linewidth=1.3,
                label=f"Mean gap ≈ {q_scores['GAP'].mean():.1f} pts")
    ax2.set_ylabel("Score Gap (Q4 − Q1)")
    ax2.set_xlabel("Year")
    ax2.set_title("Score Gap Between High- and Low-Spending States")
    ax2.legend()
    fig.suptitle("Does High Spending Translate to Higher Scores?", fontsize=15, y=1.01)
    fig.tight_layout()
    save(fig, "04j_score_gap_spending_quartile.png")


# =============================================================================
# DONE
# =============================================================================
print("\n" + "="*70)
print("EDA COMPLETE")
print("All figures saved to:  outputs/figures/eda/")
print("="*70 + "\n")


SECTION 4A — GEOSPATIAL ANALYSIS

── Fig 4-A  Per-pupil expenditure map (latest year)
  ✓  Saved  outputs/figures/eda/04a_geo_expenditure_latest.png
── Fig 4-B  NAEP composite score map (latest year)
  ✓  Saved  outputs/figures/eda/04b_geo_naep_score_latest.png
── Fig 4-C  Change in per-pupil spending map
  ✓  Saved  outputs/figures/eda/04c_geo_expenditure_change.png
── Fig 4-D  Federal revenue dependency map
  ✓  Saved  outputs/figures/eda/04d_geo_federal_dependency.png
── Fig 4-E  Spending efficiency map
  ✓  Saved  outputs/figures/eda/04e_geo_spending_efficiency.png

SECTION 4B — MEAN-REVERSION & CONVERGENCE

── Fig 4-F  Sigma-convergence in per-pupil spending
  ✓  Saved  outputs/figures/eda/04f_sigma_convergence.png
── Fig 4-G  Beta-convergence in per-pupil spending
  ✓  Saved  outputs/figures/eda/04g_beta_convergence.png
── Fig 4-H  Mean-reversion speed — deviation half-lives
  ✓  Saved  outputs/figures/eda/04h_mean_reversion_halflives.png
   Median half-life ≈ 2.1 years  across 